In [1]:
# import cPickle as pickle
import ast
import pickle
import re

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [2]:
import findspark

findspark.init()

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


spark = SparkSession\
        .builder\
        .appName("TrainingOnCombinedSummary")\
        .config('spark.sql.warehouse.dir','file:///home/sudipto21048867/Pyspark/spark-warehouse')\
        .config("spark.sql.caseSensitive","true")\
        .getOrCreate()

In [6]:
data = json.load(open('C:/Users/sudsaha1/Documents/My Received Files/TransformedSummary.json'))

TransactionDf_pd = [item for items in data.values() for item in items]
TransactionDf_pd=pd.DataFrame(TransactionDf_pd) 


TransactionDf_pd.head()


,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime
0,32353522,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-18T20:21:50-08:00,Credit,Booked,100211,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-18T20:21:50-08:00
1,32353524,"{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-18T20:21:50-08:00,Debit,Booked,100212,"3818 17NOV16 , DEBENHAMS , CROYDON GB",2016-11-18T20:21:50-08:00
2,32353526,"{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100213,"3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB",2016-11-23T20:21:50-08:00
3,32353528,"{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100214,"3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB",2016-11-23T20:21:50-08:00
4,32353530,"{""Amount"": ""4.99"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-24T20:21:50-08:00,Credit,Booked,100215,"3818 23NOV16 , LIDL UK , LONDON GB , REFUND",2016-11-24T20:21:50-08:00


In [7]:
TransactionDf=spark.createDataFrame(TransactionDf_pd)
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

,0,1,2,3,4
AccountId,32353522,32353524,32353526,32353528,32353530
Amount,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Amount"": ""4.99"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""OTR"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }"
BookingDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00
CreditDebitIndicator,Credit,Debit,Debit,Debit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100211,100212,100213,100214,100215
TransactionInformation,"CALL REF.NO. 0000 , FROM A/C 10111778","3818 17NOV16 , DEBENHAMS , CROYDON GB","3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB","3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB","3818 23NOV16 , LIDL UK , LONDON GB , REFUND"
ValueDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00


In [8]:
TransactionDf_pd.dtypes

AccountId                  int64
Amount                    object
BankTransactionCode       object
BookingDateTime           object
CreditDebitIndicator      object
Status                    object
TransactionId              int64
TransactionInformation    object
ValueDateTime             object
dtype: object

In [9]:
#sc = spark.sparkContext
#Transaction_rdd = sc.textFile("D:/Llyods/BASE_Data/CombinedSummary.csv")
#Transaction_rdd.take(5)
TransactionDf.dtypes

[('AccountId', 'bigint'),
 ('Amount', 'string'),
 ('BankTransactionCode', 'string'),
 ('BookingDateTime', 'string'),
 ('CreditDebitIndicator', 'string'),
 ('Status', 'string'),
 ('TransactionId', 'bigint'),
 ('TransactionInformation', 'string'),
 ('ValueDateTime', 'string')]

In [10]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.functions import expr
from pyspark.sql import SQLContext, Row

In [11]:
TransactionDf_pd['AmountInGBP'] = TransactionDf_pd['Amount'].map(lambda x: ast.literal_eval(x)['Amount']).astype(np.float)
TransactionDf_pd['Type'] = TransactionDf_pd['BankTransactionCode'].map(lambda x: ast.literal_eval(x[:-1] + "\"\"" + '}')['Code'])

In [12]:
TransactionDf_pd.head()

,AccountId,Amount,BankTransactionCode,BookingDateTime,CreditDebitIndicator,Status,TransactionId,TransactionInformation,ValueDateTime,AmountInGBP,Type
0,32353522,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Code"": ""OTR"","" SubCode"": }",2016-11-18T20:21:50-08:00,Credit,Booked,100211,"CALL REF.NO. 0000 , FROM A/C 10111778",2016-11-18T20:21:50-08:00,100.00,OTR
1,32353524,"{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-18T20:21:50-08:00,Debit,Booked,100212,"3818 17NOV16 , DEBENHAMS , CROYDON GB",2016-11-18T20:21:50-08:00,50.00,POS
2,32353526,"{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100213,"3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB",2016-11-23T20:21:50-08:00,2.00,POS
3,32353528,"{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-23T20:21:50-08:00,Debit,Booked,100214,"3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB",2016-11-23T20:21:50-08:00,2.40,POS
4,32353530,"{""Amount"": ""4.99"", ""Currency"": ""GBP""}","{""Code"": ""POS"","" SubCode"": }",2016-11-24T20:21:50-08:00,Credit,Booked,100215,"3818 23NOV16 , LIDL UK , LONDON GB , REFUND",2016-11-24T20:21:50-08:00,4.99,POS


In [13]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf

In [14]:

AmountInGBP = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())

Type = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())

def string_to_float(x):
    return float(x)

udfstring_to_float = udf(string_to_float, FloatType())
#paid_out_cleaned = udf(lambda y: y.replace(',', '') if y!='-' else '0', StringType())

In [15]:

TransactionDf = TransactionDf.withColumn('AmountInGBP',udfstring_to_float(AmountInGBP(TransactionDf['Amount']))).\
                              withColumn('Type',Type(TransactionDf['BankTransactionCode']))

In [16]:
TransactionDf.printSchema()

root
 |-- AccountId: long (nullable = true)
 |-- Amount: string (nullable = true)
 |-- BankTransactionCode: string (nullable = true)
 |-- BookingDateTime: string (nullable = true)
 |-- CreditDebitIndicator: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- TransactionId: long (nullable = true)
 |-- TransactionInformation: string (nullable = true)
 |-- ValueDateTime: string (nullable = true)
 |-- AmountInGBP: float (nullable = true)
 |-- Type: string (nullable = true)



In [17]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

,0,1,2,3,4
AccountId,32353522,32353524,32353526,32353528,32353530
Amount,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Amount"": ""4.99"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""OTR"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }"
BookingDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00
CreditDebitIndicator,Credit,Debit,Debit,Debit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100211,100212,100213,100214,100215
TransactionInformation,"CALL REF.NO. 0000 , FROM A/C 10111778","3818 17NOV16 , DEBENHAMS , CROYDON GB","3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB","3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB","3818 23NOV16 , LIDL UK , LONDON GB , REFUND"
ValueDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00
AmountInGBP,100,50,2,2.4,4.99


In [18]:
import numpy as np



def BinningData(TransactionDf,bins):
    max_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.max()[0] 
    min_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.min()[0] -1  ##??
    custom_bucket_array = np.linspace(min_val, max_val, 4)
    cut_points = list(custom_bucket_array)
    return cut_points

In [19]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=BinningData(TransactionDf,4),inputCol="AmountInGBP", outputCol="AmountInGBP_bin")
TransactionDf = bucketizer.transform(TransactionDf)

#TransactionDf.show()

In [20]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

,0,1,2,3,4
AccountId,32353522,32353524,32353526,32353528,32353530
Amount,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Amount"": ""4.99"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""OTR"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }"
BookingDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00
CreditDebitIndicator,Credit,Debit,Debit,Debit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100211,100212,100213,100214,100215
TransactionInformation,"CALL REF.NO. 0000 , FROM A/C 10111778","3818 17NOV16 , DEBENHAMS , CROYDON GB","3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB","3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB","3818 23NOV16 , LIDL UK , LONDON GB , REFUND"
ValueDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00
AmountInGBP,100,50,2,2.4,4.99


In [21]:
t = {0:"low", 1: "medium", 2:"high"}
map_bins = udf(lambda x: t[x], StringType())
TransactionDf = TransactionDf.withColumn("AmountInGBP_bin", map_bins("AmountInGBP_bin"))

In [22]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

,0,1,2,3,4
AccountId,32353522,32353524,32353526,32353528,32353530
Amount,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Amount"": ""50"", ""Currency"": ""GBP""}","{""Amount"": ""2"", ""Currency"": ""GBP""}","{""Amount"": ""2.4"", ""Currency"": ""GBP""}","{""Amount"": ""4.99"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""OTR"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }"
BookingDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00
CreditDebitIndicator,Credit,Debit,Debit,Debit,Credit
Status,Booked,Booked,Booked,Booked,Booked
TransactionId,100211,100212,100213,100214,100215
TransactionInformation,"CALL REF.NO. 0000 , FROM A/C 10111778","3818 17NOV16 , DEBENHAMS , CROYDON GB","3818 22NOV16 C , COOP GROUP FOOD , CROYDON GB","3818 22NOV16 C , TFL.GOV.UK/CP , TFL TRAVEL CH GB","3818 23NOV16 , LIDL UK , LONDON GB , REFUND"
ValueDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-23T20:21:50-08:00,2016-11-24T20:21:50-08:00
AmountInGBP,100,50,2,2.4,4.99


In [23]:
# Get Merchant info from Description
def getMerchfromDesc(tranType, tranDesc):
    if tranType == 'POS':
        parsed = tranDesc.split(', ')
        lenPar = len(parsed)
        if parsed[lenPar - 1] == 'REFUND':
            merch = parsed[1:-2]
            return ('').join(merch) + 'REFUND'
        else:
            merch = parsed[1:-1]
            return ('').join(merch)
    else:
        return tranDesc
    
import re    
    
# Clean Merchant info
def cleanMerchant(txt):
    txt = re.sub(r'[^a-zA-Z\b]', ' ', str(txt).upper())
    return " ".join(txt.split())    

In [38]:
parse_merchant = udf(getMerchfromDesc , StringType())
parse_cleanMerchant = udf(cleanMerchant , StringType())
#Type = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())



TransactionDf = TransactionDf.withColumn('Merchant',parse_merchant(TransactionDf['Type'],TransactionDf['TransactionInformation']))
#                              .withColumn('cleanedMerchant',parse_cleanMerchant(TransactionDf['Merchant']))
TransactionDf = TransactionDf.withColumn('cleanedMerchant',parse_cleanMerchant(TransactionDf['Merchant']))

In [40]:
pd.DataFrame(TransactionDf.take(2),columns= TransactionDf.columns).transpose()

,0,1
AccountId,32353522,32353524
Amount,"{""Amount"": ""100"", ""Currency"": ""GBP""}","{""Amount"": ""50"", ""Currency"": ""GBP""}"
BankTransactionCode,"{""Code"": ""OTR"","" SubCode"": }","{""Code"": ""POS"","" SubCode"": }"
BookingDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00
CreditDebitIndicator,Credit,Debit
Status,Booked,Booked
TransactionId,100211,100212
TransactionInformation,"CALL REF.NO. 0000 , FROM A/C 10111778","3818 17NOV16 , DEBENHAMS , CROYDON GB"
ValueDateTime,2016-11-18T20:21:50-08:00,2016-11-18T20:21:50-08:00
AmountInGBP,100,50


In [34]:
def ConcatCategory(category,Detailed_Category):
    return category + ', ' + Detailed_Category


ConcatCategory = udf(ConcatCategory,StringType())

In [35]:
TransactionDf = TransactionDf.withColumn('ConcatCategory',ConcatCategory(TransactionDf['category'],TransactionDf['Detailed Category']))

AnalysisException: 'Cannot resolve column name "category" among (AccountId, Amount, BankTransactionCode, BookingDateTime, CreditDebitIndicator, Status, TransactionId, TransactionInformation, ValueDateTime, AmountInGBP, Type, AmountInGBP_bin, Merchant, cleanedMerchant);'

In [37]:
TransactionDf.take(1)

[Row(AccountId=32353522, Amount='{"Amount": "100", "Currency": "GBP"}', BankTransactionCode='{"Code":  "OTR"," SubCode": }', BookingDateTime='2016-11-18T20:21:50-08:00', CreditDebitIndicator='Credit', Status='Booked', TransactionId=100211, TransactionInformation='CALL REF.NO. 0000 , FROM A/C 10111778', ValueDateTime='2016-11-18T20:21:50-08:00', AmountInGBP=100.0, Type='OTR', AmountInGBP_bin='low', Merchant='CALL REF.NO. 0000 , FROM A/C 10111778', cleanedMerchant='CALL REF NO FROM A C')]

In [41]:
TransactionDf_pd['Merchant'] = TransactionDf_pd.apply(lambda row: getMerchfromDesc(row['Type'], row['TransactionInformation']), axis=1)
TransactionDf_pd['cleanedMerchant'] = TransactionDf_pd['Merchant'].map(lambda x: cleanMerchant(x))

TransactionDf_pd['ConcatCategory'] = TransactionDf_pd['category'] + ', ' + TransactionDf_pd['Detailed Category']
# len(TransactionDf['ConcatCategory'].unique())

#TransactionDf_pd['Predictorcolumn'] = TransactionDf_pd['CreditDebitIndicator'] + ' ' + TransactionDf_pd['cleanedMerchant'] + ' ' + TransactionDf_pd['Type'] + ' ' + TransactionDf_pd['AmountInGBP_bin'].astype(str)

TransactionDf_pd.head().transpose()

KeyError: 'category'

In [ ]:
AmountInGBP = udf(lambda x: float(ast.literal_eval(x)['Amount']) , FloatType())
getElement = udf(lambda v,i: v[i], StringType())

In [ ]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.functions import *

In [ ]:
TransactionDf = TransactionDf.withColumn('AmountInGBP',AmountInGBP(TransactionDf['Amount']))\
                             .withColumn('Type',getElement('BankTransactionCode',lit(0)))

In [ ]:
TransactionDf.first()

In [ ]:
#Python
TransactionDf_pd['AmountInGBP'] = TransactionDf_pd['Amount'].map(lambda x: ast.literal_eval(x)['Amount']).astype(np.float)
TransactionDf_pd['Type'] = TransactionDf_pd['BankTransactionCode'].map(lambda x: ast.literal_eval(x[:-1] + "\"\"" + '}')['Code'])


In [ ]:
import numpy as np



def BinningData(TransactionDf,bins):
    max_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.max()[0] 
    min_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.min()[0] -1  ##??
    custom_bucket_array = np.linspace(min_val, max_val, 4)
    cut_points = list(custom_bucket_array)
    return cut_points